In [1]:
!nvidia-smi

Mon Jun 10 23:32:26 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              44W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
!pip install -U accelerate==0.29.3
!pip install peft==0.10.0
!pip install bitsandbytes==0.43.1
!pip install transformers==4.40.1
!pip install trl==0.8.6
!pip install datasets==2.19.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [3]:
import os
import torch
from datasets import load_dataset

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

import huggingface_hub
huggingface_hub.login("hf_token")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
# Hugging Face Basic Model 한국어 모델
# base_model = "teddylee777/Llama-3-Open-Ko-8B-gguf"  # 테디님의 Llama3 한국어 파인튜닝 모델
base_model = "beomi/Llama-3-Open-Ko-8B"               # beomi님의 Llama3 한국어 파인튜닝 모델

# baemin_dataset_simple.json
dataset = "/content/dataset"

# 새로운 모델 이름
new_model = "llama3-sft"

In [6]:
dataset = load_dataset(dataset, split="train")

# 데이터 확인
print(len(dataset))
print(dataset[0])

Generating train split: 0 examples [00:00, ? examples/s]

3994
{'text': '<s>[INST] Q. 일용직인데 노동청에 신고했어요 [/INST] 근로자의 해고와 관련하여 근로기준법 위반 등에 대해 문의한 것으로 보여집니다. 1. 원칙적으로 일용직 근로자라도 근로계약서를 작성하여야 합니다. 어떠한 경위에서 근로계약서를 작성하지 아니하였는지는 분명하지는 않지만, 위 내용대로 사용자가 근로계약서 작성을 지속적으로 요청하였음에도 근로자가 일방적으로 거부하여 작성할 수 없었다면 위 내용을 입증할 수 있는 자료를 구비하여 근로계약서 미작성과 관련한 진정 제기 시 고의성이 없는 부분으로 대응하시기 바랍니다. 2. 실질이 일용직인지를 살펴보아야 합니다. 해당 근로자의 최초 입사일, 근로제공방법, 근로제공형태, 근로제공기간을 모두 살펴보아야 합니다. 3. 사용자가 전달한 의사가 해고의사인지, 근로관계 유지를 이유로 일시적인 휴직의 의사인지부터 명확하게 살펴보아야 합니다. </s>'}


In [7]:
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install -qqq flash-attn
    attn_implementation = "flash_attention_2"
    torch_dtype = torch.bfloat16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float16

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 39.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 6.7 MB/s eta 0:00:00


In [8]:
# QLoRA config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=False,
)

In [9]:
# llama 데이터 로드
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/3.00G [00:00<?, ?B/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/2.94G [00:00<?, ?B/s]

...

model-00004-of-00006.safetensors:   0%|          | 0.00/2.94G [00:00<?, ?B/s]

model-00005-of-00006.safetensors:   0%|          | 0.00/2.94G [00:00<?, ?B/s]

model-00006-of-00006.safetensors:   0%|          | 0.00/1.29G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [10]:
# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(
              base_model,
              trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [12]:
training_params = TrainingArguments(
    output_dir="/results",
    num_train_epochs = 1, #epoch는 1로 설정
    max_steps=5000, #max_steps을 5000으로 설정
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    optim="paged_adamw_8bit",
    warmup_steps=0.03,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=100,
    push_to_hub=False,
    report_to='none',
)

In [13]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    dataset_text_field="text",
    max_seq_length=256,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

Map:   0%|          | 0/3994 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer.train()

Step,Training Loss
100,1.904600
200,1.750600
300,1.692100
400,1.689000
500,1.632000
600,1.663900
700,1.617900
800,1.603600
900,1.623000
1000,1.621400


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in

Step,Training Loss
100,1.904600
200,1.750600
300,1.692100
400,1.689000
500,1.632000
600,1.663900
700,1.617900
800,1.603600
900,1.623000
1000,1.621400


In [20]:
trainer.save_model(new_model)

In [22]:
import zipfile
import shutil
from google.colab import files

# 압축할 폴더 이름
folder_name = "llama3-sft"  # Data Augmentation 적용 o

# 생성될 ZIP 파일 이름
zip_file_name = "llama3-sfttrainer-1epoch.zip" # Data Augmentation 적용 o

# 폴더를 ZIP 파일로 압축
shutil.make_archive(zip_file_name[:-4], 'zip', folder_name)

# ZIP 파일을 로컬로 다운로드
files.download(zip_file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [19]:
logging.set_verbosity(logging.CRITICAL)

prompt = "알바생이 3일 일하고 그만뒀는데 주휴수당을 줘야 하나요?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] 알바생이 3일 일하고 그만뒀는데 주휴수당을 줘야 하나요? [/INST] 주휴수당 지급의무에 대한 문의를 주셨습니다. (1) 주휴수당은 주 소정근로시간이 15시간 이상인 근로자에게 적용되는 제도이므로 주 소정근로시간이 15시간 미만인 근로자에게는 적용하지 않아도 됩니다. (2) 위 근로자의 경우 주휴수당은 지급하지 않아도 될 것입니다. </ 1. 주휴수당 지급의무에 대한 문의를 주셨습니다. (1) 주휴수당은 주 소정근로시간이 15시간 이상인 근로자에게 적용되는 제도이므로 주 소정근로시간이 15시간 미만인 근로자에게는 적용하지 않아도 됩니다
